## Exercise 1

Use the following code to explore l1 and l2 regularization.  Try different combinations of l1, l2, and alpha.  What works the best?

In [1]:
#Setting up some data

import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

In [3]:
# You'll need to modify this to try different regularizers.  To make your life easier,
# you might want to create a function to generate your model

tf.random.set_seed(42) 

from functools import partial
def bdfunction(norm,alpha):
    if norm==1:
        return tf.keras.regularizers.l1(alpha)
    else:
        return tf.keras.regularizers.l2(alpha)

acc_list=[]

for norm in [1,2]:
    for alpha in [0.01,0.1,0.5]:
        RegularizedDense = partial(tf.keras.layers.Dense,
                                activation="relu",
                                kernel_initializer="he_normal",
                                kernel_regularizer=bdfunction(norm,alpha))

        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=[28, 28]),
            RegularizedDense(100),
            RegularizedDense(100),
            RegularizedDense(10, activation="softmax")
        ])
        optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.02)
        model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
        history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))
        test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
        acc_list.append(test_acc)

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 8.3345 - accuracy: 0.6332 - val_loss: 1.9222 - val_accuracy: 0.6596
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.6999 - accuracy: 0.6597 - val_loss: 1.5453 - val_accuracy: 0.6950
Epoch 3/10
1719/1719 [==============================] - 3s 1ms/step - loss: 1.5351 - accuracy: 0.6778 - val_loss: 1.5106 - val_accuracy: 0.6778
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.4928 - accuracy: 0.6815 - val_loss: 1.4602 - val_accuracy: 0.7138
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.4651 - accuracy: 0.6867 - val_loss: 1.4840 - val_accuracy: 0.6888
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.4405 - accuracy: 0.6927 - val_loss: 1.4581 - val_accuracy: 0.6844
Epoch 7/10
1719/1719 [==============================] - 3s 1ms/step - loss: 1.4283 - accuracy: 0.6943 - val_loss: 1.3844 - val_accuracy:

In [4]:
acc_list

[0.7300999760627747,
 0.10000000149011612,
 0.10000000149011612,
 0.8134999871253967,
 0.5468000173568726,
 0.10000000149011612]

## Exercise 2

1. Build two networks for processing the MNIST Fashion data.  Both networks should have three dense layers (but may also have dropout layers or activation function layers) with 100 neurons.

    1.  A self-normalizing neural network with dropout.
    2.  A network without normalization but using dropout.
    3.  A network with BatchNormalization.

Try each with either a 1cycle or performance schedule.  Which works the best?

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.utils import normalize
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
# Clear previous TensorFlow session
K.clear_session()

# Load the dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data
X_train = X_train.reshape((-1, 28*28))
X_test = X_test.reshape((-1, 28*28))

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [29]:
def self_normalizing_nn_with_dropout():
    model = models.Sequential()
    model.add(layers.Dense(100, activation='selu', kernel_initializer='lecun_normal', input_shape=(28*28,)))
    model.add(layers.AlphaDropout(0.25))
    model.add(layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.25))
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [30]:
def nn_with_dropout():
    model = models.Sequential()
    model.add(layers.Dense(100, activation='relu', input_shape=(28*28,)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(10, activation='softmax'))
    return model


In [31]:
def nn_with_batch_normalization():
    model = models.Sequential()
    model.add(layers.Dense(100, activation='relu', input_shape=(28*28,)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))
    return model


In [32]:
def lr_schedule_1cycle(epoch, lr):
    max_lr = 0.1
    end_percentage = 0.1
    total_steps = 5
    anneal_steps = int(total_steps * end_percentage)
    steps = epoch % total_steps
    if steps == 0 and epoch != 0:
        return lr * 10
    if steps < anneal_steps:
        return (max_lr - lr) / anneal_steps * steps + lr
    else:
        return (lr - max_lr) / (total_steps - anneal_steps) * (steps - anneal_steps) + max_lr


In [27]:
def lr_schedule_performance(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


In [33]:
models_to_train = [
    ('Self-Normalizing NN with Dropout', self_normalizing_nn_with_dropout),
    ('NN with Dropout', nn_with_dropout),
    ('NN with Batch Normalization', nn_with_batch_normalization)
]

schedules = [
    ('1cycle', lr_schedule_1cycle),
    ('Performance', lr_schedule_performance)
]

results = {}

for model_name, model_fn in models_to_train:
    for schedule_name, lr_schedule in schedules:
        model = model_fn()
        model.compile(optimizer=SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
        
        if schedule_name == '1cycle':
            lr_callback = LearningRateScheduler(lr_schedule)
            history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), callbacks=[lr_callback])
        else:
            lr_callback = callbacks.LearningRateScheduler(lr_schedule, verbose=1)
            history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), callbacks=[lr_callback])
        
        results[(model_name, schedule_name)] = history.history['val_accuracy']

Epoch 1/10
750/750 [==============================] - 3s 3ms/step - loss: 0.8132 - accuracy: 0.6945 - val_loss: 0.5912 - val_accuracy: 0.8069 - lr: 0.1000
Epoch 2/10
750/750 [==============================] - 2s 3ms/step - loss: 0.5819 - accuracy: 0.7878 - val_loss: 0.6114 - val_accuracy: 0.8004 - lr: 0.1000
Epoch 3/10
750/750 [==============================] - 2s 2ms/step - loss: 0.5346 - accuracy: 0.8054 - val_loss: 0.5248 - val_accuracy: 0.8308 - lr: 0.1000
Epoch 4/10
750/750 [==============================] - 2s 3ms/step - loss: 0.5077 - accuracy: 0.8164 - val_loss: 0.5010 - val_accuracy: 0.8358 - lr: 0.1000
Epoch 5/10
750/750 [==============================] - 3s 4ms/step - loss: 0.4852 - accuracy: 0.8242 - val_loss: 0.5192 - val_accuracy: 0.8336 - lr: 0.1000
Epoch 6/10
750/750 [==============================] - 2s 3ms/step - loss: 228881968.0000 - accuracy: 0.1030 - val_loss: 4844734.5000 - val_accuracy: 0.0997 - lr: 1.0000
Epoch 7/10
750/750 [==============================] - 2s


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 1.4740 - accuracy: 0.4595 - val_loss: 0.7803 - val_accuracy: 0.7061 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/10
750/750 [==============================] - 1s 2ms/step - loss: 0.9517 - accuracy: 0.6344 - val_loss: 0.7227 - val_accuracy: 0.7352 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/10
750/750 [==============================] - 2s 2ms/step - loss: 0.8200 - accuracy: 0.6877 - val_loss: 0.6966 - val_accuracy: 0.7535 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 4/10
750/750 [==============================] - 2s 2ms/step - loss: 0.7536 - accuracy: 0.7164 - val_loss: 0.6764 - val_accuracy: 0.7645 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.00999

Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 0.7226 - accuracy: 0.7420 - val_loss: 0.5017 - val_accuracy: 0.8146 - lr: 0.1000
Epoch 2/10
750/750 [==============================] - 1s 2ms/step - loss: 0.5087 - accuracy: 0.8186 - val_loss: 0.4715 - val_accuracy: 0.8280 - lr: 0.1000
Epoch 3/10
750/750 [==============================] - 1s 2ms/step - loss: 0.4598 - accuracy: 0.8337 - val_loss: 0.3901 - val_accuracy: 0.8538 - lr: 0.1000
Epoch 4/10
750/750 [==============================] - 1s 2ms/step - loss: 0.4315 - accuracy: 0.8451 - val_loss: 0.3943 - val_accuracy: 0.8553 - lr: 0.1000
Epoch 5/10
750/750 [==============================] - 1s 2ms/step - loss: 0.4095 - accuracy: 0.8514 - val_loss: 0.3774 - val_accuracy: 0.8577 - lr: 0.1000
Epoch 6/10
750/750 [==============================] - 1s 2ms/step - loss: 504640672.0000 - accuracy: 0.1015 - val_loss: 2.3054 - val_accuracy: 0.0966 - lr: 1.0000
Epoch 7/10
750/750 [==============================] - 1s 2ms/s


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 1.2010 - accuracy: 0.5864 - val_loss: 0.7330 - val_accuracy: 0.7531 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/10
750/750 [==============================] - 1s 2ms/step - loss: 0.7820 - accuracy: 0.7268 - val_loss: 0.6184 - val_accuracy: 0.7889 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/10
750/750 [==============================] - 1s 2ms/step - loss: 0.6820 - accuracy: 0.7623 - val_loss: 0.5602 - val_accuracy: 0.8087 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 4/10
750/750 [==============================] - 1s 2ms/step - loss: 0.6210 - accuracy: 0.7842 - val_loss: 0.5241 - val_accuracy: 0.8184 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.00999

Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 0.4922 - accuracy: 0.8209 - val_loss: 0.5047 - val_accuracy: 0.8106 - lr: 0.1000
Epoch 2/10
750/750 [==============================] - 2s 2ms/step - loss: 0.3822 - accuracy: 0.8616 - val_loss: 0.4275 - val_accuracy: 0.8433 - lr: 0.1000
Epoch 3/10
750/750 [==============================] - 1s 2ms/step - loss: 0.3537 - accuracy: 0.8704 - val_loss: 0.3725 - val_accuracy: 0.8653 - lr: 0.1000
Epoch 4/10
750/750 [==============================] - 1s 2ms/step - loss: 0.3328 - accuracy: 0.8766 - val_loss: 0.3989 - val_accuracy: 0.8541 - lr: 0.1000
Epoch 5/10
750/750 [==============================] - 1s 2ms/step - loss: 0.3085 - accuracy: 0.8856 - val_loss: 0.4102 - val_accuracy: 0.8566 - lr: 0.1000
Epoch 6/10
750/750 [==============================] - 2s 2ms/step - loss: 0.5001 - accuracy: 0.8209 - val_loss: 0.4612 - val_accuracy: 0.8323 - lr: 1.0000
Epoch 7/10
750/750 [==============================] - 1s 2ms/step - lo


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 0.6127 - accuracy: 0.7920 - val_loss: 0.4765 - val_accuracy: 0.8341 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 2/10
750/750 [==============================] - 1s 2ms/step - loss: 0.4346 - accuracy: 0.8461 - val_loss: 0.4056 - val_accuracy: 0.8575 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 3/10
750/750 [==============================] - 2s 2ms/step - loss: 0.3931 - accuracy: 0.8607 - val_loss: 0.3878 - val_accuracy: 0.8592 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 4/10
750/750 [==============================] - 1s 2ms/step - loss: 0.3685 - accuracy: 0.8677 - val_loss: 0.3866 - val_accuracy: 0.8609 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.00999

In [42]:
import numpy as np
for item in results:
    print(item,np.max(results[item]))

('Self-Normalizing NN with Dropout', '1cycle') 0.8358333110809326
('Self-Normalizing NN with Dropout', 'Performance') 0.7989166378974915
('NN with Dropout', '1cycle') 0.8576666712760925
('NN with Dropout', 'Performance') 0.8424999713897705
('NN with Batch Normalization', '1cycle') 0.8818333148956299
('NN with Batch Normalization', 'Performance') 0.8790000081062317


## Exercise 3

Use hyperparameter tuning with the Fashion MNIST data to explore the following parameters:

- Dropout rate
- Number of layers (2-4)
- Number of neurons per layer (64 - 128)

Use Bayesian optimization and tensorboard to examine your results.


In [59]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime

# Load and preprocess Fashion MNIST dataset
(X_train, y_train), (X_val, y_val) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

# Define the neural network architecture
class MyModel:
    def __init__(self):
        pass

    def fit(self, X, y, dropout_rate, num_layers, num_neurons):
        model = Sequential()
        model.add(Flatten(input_shape=(28, 28)))
        for _ in range(num_layers):
            model.add(Dense(num_neurons, activation='relu'))
            model.add(Dropout(dropout_rate))
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.model = model
        self.history = self.model.fit(X, y, epochs=3, verbose=0)  # Adjust epochs as needed

    def score(self, X, y):
        return self.model.evaluate(X, y, verbose=0)[1]

# Define the hyperparameter search space
search_space = {
    'dropout_rate': (0.1, 0.5),
    'num_layers': (2, 4),
    'num_neurons': (64, 128)
}

# Set up Bayesian optimization
bayes_search = BayesSearchCV(
    estimator=MyModel(),
    search_spaces=search_space,
    n_iter=20,  # Number of iterations for the search
    cv=StratifiedKFold(n_splits=3),
    n_jobs=-1,
    verbose=1
)

# Set up TensorBoard
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir) 
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2) 

# Perform hyperparameter optimization
bayes_search.fit(X_train, y_train, callback=[tensorboard_callback,early_stopping_cb])

# Get the best hyperparameters
best_params = bayes_search.best_params_
print("Best hyperparameters:", best_params)


ValueError: callback should be either a callable or a list of callables.